#Funções de perdas


In [1]:
import torch
from torch import nn
if torch.cuda.is_available():
  print("GPU ")
else :
  print("outra coisa")



GPU 


In [3]:
from sklearn import datasets

wine = datasets.load_wine()
data = wine.data
target = wine.target
print(data.shape, target.shape)
print(wine.feature_names, wine.target_names)

(178, 13) (178,)
['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline'] ['class_0' 'class_1' 'class_2']
